In [ ]:
# default_exp core

# s4refsimtool

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export

import os
base_folder = os.environ.get("S4REFSIMTOOL_INPUT",
                             "/global/cscratch1/sd/keskital/s4sim/reference_tool/out/")

In [ ]:
# export

simulations_observing_efficiency = dict(
    pole=dict(SAT=46.29 / 100, LAT=37.23 / 100), chile=dict()
)

In [ ]:
#export


def get_telescope(channel):
    """Identify if a channel is in SAT or LAT

    Parameters
    ----------
    channel : str
        channel tag, e.g. MFH1

    Returns
    -------
    telescope : str
        'SAT' or 'LAT'
    """
    return channel[-2] + "AT"

In [ ]:
# export

import h5py

s4 = h5py.File("cmbs4_tophat.h5", "r")


def parse_channels(channels):
    if channels in ["SAT", "LAT"]:
        channels = [ch for ch in s4.keys() if s4[ch].attrs["telescope"] == channels]
    elif channels in ["all", None]:
        channels = list(s4.keys())
    elif isinstance(channels, str):
        channels = channels.split(",")
    return channels

In [ ]:
assert parse_channels("LFL1") == ["LFL1"]
assert parse_channels(["LFL1"]) == ["LFL1"]
assert parse_channels("LFL1,LFL2") == ["LFL1", "LFL2"]
assert parse_channels("LAT") == ['HFL1', 'HFL2', 'LFL1', 'LFL2', 'MFL1', 'MFL2', 'ULFL1']
assert parse_channels("all") == list(s4.keys())